### Tarea 2: Tratamiento Automático del Lenguaje

nombre: Felipe Córdova

El dataset se encuentra en: https://drive.google.com/drive/folders/1JS-Afzz4cKzE7VEO1a8KYVfK2LbdlAaB

In [11]:
# Importar librerías necesarias
import chromadb
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
import pandas as pd
from datetime import datetime

# Configuración inicial
api_key = "tu_api_key"
llm = OpenAI(temperature=0.7, openai_api_key=api_key)

# Crear cliente de ChromaDB
client = chromadb.PersistentClient(path="./chromadb_data")
collection = client.get_or_create_collection("hidrogeno_verde")

# Cargar el dataset
df = pd.read_csv('hidrogeno_verde.csv')

# Convertir la columna 'date' al tipo datetime con el formato correcto
df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y @ %H:%M:%S.%f')

# NO convertir la columna 'date' a texto con strftime, mantenerla como datetime
# Ya está en formato datetime, listo para ser utilizado con .isoformat()

# Insertar datos en ChromaDB (vectorización básica por texto y título)
for index, row in df.iterrows():
    collection.add(
        documents=[row['text']],
        metadatas={'id_news': row['id_news'], 'date': row['date'].isoformat()},
        ids=[str(row['id_news'])]
    )

Insert of existing embedding ID: 54.272.642
Add of existing embedding ID: 54.272.642
Insert of existing embedding ID: 54.270.980
Add of existing embedding ID: 54.270.980
Insert of existing embedding ID: 54.274.158
Add of existing embedding ID: 54.274.158
Insert of existing embedding ID: 54.293.366
Add of existing embedding ID: 54.293.366
Insert of existing embedding ID: 54.293.490
Add of existing embedding ID: 54.293.490
Insert of existing embedding ID: 54.182.703
Add of existing embedding ID: 54.182.703
Insert of existing embedding ID: 54.186.155
Add of existing embedding ID: 54.186.155
Insert of existing embedding ID: 54.294.705
Add of existing embedding ID: 54.294.705
Insert of existing embedding ID: 53.989.089
Add of existing embedding ID: 53.989.089
Insert of existing embedding ID: 54.295.271
Add of existing embedding ID: 54.295.271
Insert of existing embedding ID: 53.877.936
Add of existing embedding ID: 53.877.936
Insert of existing embedding ID: 54.295.980
Add of existing embed

In [12]:
df

,date,id_news,text,title,url,polarity_score,polarity_label,media_outlet,country
0,2024-10-19 21:00:00,54.272.642,"Santiago, octubre de 2024 – Como parte de la c...",Ausenco impulsa la transición energética a tra...,https://tierramarillano.cl/2024/10/20/ausenco-...,-,-,tierraamarillano,chile
1,2024-10-17 21:00:00,54.270.980,"Claudio Seebach, una voz respetada en el sect...","Claudio Seebach aterriza la ""realidad"" del hid...",https://www.emol.com/noticias/Economia/2024/10...,-,-,emol,chile
2,2024-10-16 21:00:00,54.274.158,"En septiembre, el Presidente Gabriel Boric i...",Hidrógeno verde: La industria estratégica para...,https://www.emol.com/noticias/Economia/2024/10...,-,-,emol,chile
3,2024-10-14 21:00:00,54.293.366,– Eliana Ximena Oyarzún Nancuante La División ...,Subpesca solicita a la industria del hidrógeno...,https://laprensaaustral.cl/2024/10/15/subpesca...,-,-,laprensaaustral,chile
4,2024-10-14 21:00:00,54.293.490,– Eliana Ximena Oyarzún Nancuante Su profunda ...,Sindicato de Trabajadores Enap llama a “aterri...,https://laprensaaustral.cl/2024/10/15/sindicat...,-,-,laprensaaustral,chile
...,...,...,...,...,...,...,...,...,...
411,2020-01-22 21:00:00,12.237.141,Hace décadas que el hidrógeno se ha promociona...,"Hidrógeno verde: Qué es, cómo se produce y cuá...",https://www.emol.com/noticias/Economia/2020/01...,"0,975",NEU,emol,chile
412,2020-01-22 21:00:00,21.971.965,La crítica situación actual del cambio climáti...,Qué es el hidrógeno verde y porqué Chile podrí...,https://www.meganoticias.cl/nacional/289162-hi...,"0,993",NEU,ahoranoticiasmega,chile
413,2020-01-21 21:00:00,12.237.509,Ante la necesidad de Chile -y el mundo- por ex...,"""Los números son extraordinarios para Chile"": ...",https://www.emol.com/noticias/Economia/2020/01...,"0,982",POS,emol,chile
414,2020-01-06 21:00:00,5.183.873,Los especialistas destacan que el hidrógeno re...,"Hidrógeno verde, la opción más eficaz para lim...",https://www.elciudadano.com/medio-ambiente/hid...,"0,997",POS,elciudadano,chile


In [26]:
# Definir funciones del sistema RAG

# 1. Buscar noticias sobre un tema entre dos fechas
def search_news_by_date_range(topic, start_date, end_date):
    from datetime import datetime

    start_date = datetime.fromisoformat(start_date)
    end_date = datetime.fromisoformat(end_date)

    # Realizar la consulta en ChromaDB
    results = collection.query(
        query_texts=[topic],
        n_results=300
    )

    # Inspeccionar estructura de metadatos
    print("Metadatas:", results.get('metadatas', None))

    # Filtrar resultados
    filtered_results = []
    if 'metadatas' in results and isinstance(results['metadatas'], list):
        for doc, metadata in zip(results['documents'], results['metadatas']):
            print("Metadata:", metadata)  # Para depuración
            if isinstance(metadata, dict) and 'date' in metadata:
                doc_date = datetime.fromisoformat(metadata['date'])
                if start_date <= doc_date <= end_date:
                    filtered_results.append(doc)
            else:
                print("Formato inesperado en metadata:", metadata)
    else:
        raise ValueError("Unexpected results format from collection.query")

    return filtered_results



# 2. Extraer menciones de personas
def extract_person_mentions(news):
    prompt = PromptTemplate(
        input_variables=["text"],
        template="Del siguiente texto, extrae las personas mencionadas: {text}"
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    mentions = []
    for doc in news:
        mentions.append(chain.run(text=doc))
    return mentions

# 3. Consultar sobre una persona
def query_person(person, context):
    prompt = PromptTemplate(
        input_variables=["persona", "contexto"],
        template=(
            "Dada la información sobre hidrógeno verde: {contexto}, "
            "responde sobre la persona {persona}. ¿Cuál es su opinión o posición? ¿Qué declaraciones ha hecho?"
        )
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(persona=person, contexto=" ".join(context))
    return response

# 4. Obtener foto de una persona (usando búsqueda externa)
def get_photo_link(person):
    prompt = PromptTemplate(
        input_variables=["persona"],
        template=(
            "Busca en Internet un enlace a una foto pública de {persona} y devuelve solo la URL de la foto, no del buscador."
        )
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain.run(persona=person)

# Ejemplo de uso

# a) Buscar noticias entre dos fechas
topic = "hidrógeno verde"
start_date = "2018-01-01"
end_date = "2024-01-10"
news = search_news_by_date_range(topic, start_date, end_date)

# b) Extraer menciones de personas
persons = extract_person_mentions(news)

# c) Consultar sobre "Juan Carlos Jobet"
person_of_interest = "Juan Carlos Jobet"
context = [doc['text'] for doc in news if person_of_interest in doc['text']]
response = query_person(person_of_interest, context)

# d) Obtener enlace a una foto
photo_link = get_photo_link(person_of_interest)

print("Noticias encontradas:", news)
print("\nPersonas mencionadas:", persons)
print(f"\nInformación sobre {person_of_interest}:", response)
print("\nEnlace a foto:", photo_link)


Metadatas: [[{'date': '2022-11-30T21:00:00', 'id_news': '51.138.043'}, {'date': '2022-08-06T20:00:00', 'id_news': '51.157.103'}, {'date': '2020-01-21T21:00:00', 'id_news': '12.237.509'}, {'date': '2024-06-25T20:00:00', 'id_news': '52.874.128'}, {'date': '2023-11-01T21:00:00', 'id_news': '52.334.767'}, {'date': '2022-06-23T20:00:00', 'id_news': '47.150.450'}, {'date': '2024-02-13T21:00:00', 'id_news': '52.555.776'}, {'date': '2021-03-14T21:00:00', 'id_news': '21.982.089'}, {'date': '2023-08-02T20:00:00', 'id_news': '52.367.312'}, {'date': '2023-08-03T20:00:00', 'id_news': '50.379.674'}, {'date': '2023-04-27T20:00:00', 'id_news': '50.336.492'}, {'date': '2023-12-22T21:00:00', 'id_news': '52.486.342'}, {'date': '2022-01-04T21:00:00', 'id_news': '21.954.952'}, {'date': '2021-01-23T21:00:00', 'id_news': '54.358.987'}, {'date': '2022-11-20T21:00:00', 'id_news': '51.139.076'}, {'date': '2020-08-22T20:00:00', 'id_news': '6.356.612'}, {'date': '2021-06-20T20:00:00', 'id_news': '54.353.284'}, {'